# 1. LM Studio 테스트

In [2]:
# Chat with an intelligent assistant in your terminal
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You allways answer in Korean."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    completion = client.chat.completions.create(
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    
    # Uncomment to see chat history
    # import json
    # gray_color = "\033[90m"
    # reset_color = "\033[0m"
    # print(f"{gray_color}\n{'-'*20} History dump {'-'*20}\n")
    # print(json.dumps(history, indent=2))
    # print(f"\n{'-'*55}\n{reset_color}")

    print()
    history.append({"role": "user", "content": input("> ")})

KeyboardInterrupt: 

# 생성 토큰수 계산

In [9]:
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

def load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", k=10):
    # Initialize the embedding model
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    
    # Load Chroma vectorstore with the embedding function
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    
    # Create retriever
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    
    return retriever

# Example usage
chroma_dir = "chroma_data_folder_final"  # Path to the Chroma directory
retriever = load_chroma_retriever_with_embedding(chroma_dir, k=10)

In [11]:
result = retriever.get_relevant_documents("아동을 대상으로 한 광고는 어떤 규제를 받나요?")
count = 0


for i in result:
    count += len(i.page_content)
print(count)

6953


In [12]:
result

[Document(metadata={'article': '제10조', 'related_references': '제2조, 제10조', 'revision_dates': '[{"year": 2016, "month": 2, "day": 3}, {"year": 2013, "month": 3, "day": 23}, {"year": 2013, "month": 7, "day": 30}, {"year": 2023, "month": 8, "day": 8}, {"year": 2013, "month": 3, "day": 23}, {"year": 2013, "month": 7, "day": 30}]', 'source': 'file/현행법령_copy/어린이 식생활안전관리 특별법(법률)(제19622호)(20240209).pdf', 'title': '광고의 제한ㆍ금지 등'}, page_content='제10조(광고의 제한ㆍ금지 등) ① 어린이 기호식품 중 고열량ㆍ저영양 식품 및 고카페인 함유 식품을 제조ㆍ가공ㆍ수\n입ㆍ유통ㆍ판매하는 자는 방송, 라디오 및 인터넷을 이용하여 식품이 아닌 장난감이나 그 밖에 어린이의 구매를 부\n추길 수 있는 물건을 무료로 제공한다는 내용이 담긴 광고를 하여서는 아니 된다. <개정 2016. 2. 3.>\n② 식품의약품안전처장은 어린이 기호식품을 제조ㆍ가공ㆍ수입ㆍ유통ㆍ판매하는 자가 「방송법」 제2조제1호가목\n의 텔레비전방송 또는 「인터넷 멀티미디어 방송사업법」 제2조제1호에 따른 인터넷 멀티미디어 방송을 이용하여 고\n열량ㆍ저영양 식품과 고카페인 함유 식품을 광고하는 경우 그 광고시간의 일부를 제한하거나 광고를 금지하게 할\n수 있다.<개정 2013. 3. 23., 2013. 7. 30., 2023. 8. 8.>\n③ 식품의약품안전처장은 제2항에 따라 광고시간의 일부를 제한하거나 광고를 금지하고자 하는 경우 그 대상이 되\n는 고열량ㆍ저영양 식품과 고카페인 함유 식품을 정하여 고시하여야 한다.<개정 2013. 3. 23., 2013. 7. 30.>\n④ 제

In [18]:
result = retriever.get_relevant_documents("도로교통법 제 20조에 대해 알려주세요.")

In [15]:
from sentence_transformers import SentenceTransformer

# 모델 로드
model_name = "jinaai/jina-embeddings-v3"  # 사용한 모델 이름
model = SentenceTransformer(model_name, trust_remote_code=True)

# 토크나이저 확인
tokenizer = model.tokenizer
print(f"사용 중인 토크나이저: {tokenizer}")
print(f"토크나이저 클래스: {type(tokenizer)}")

사용 중인 토크나이저: XLMRobertaTokenizerFast(name_or_path='jinaai/jina-embeddings-v3', vocab_size=250002, model_max_length=8194, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)
토크나이저 클래스: <class 'transformers.models.xlm_roberta.tokenization_xlm

In [17]:
# 샘플 문장
text = "안녕하세요, 오늘 날씨가 참 좋네요!"

# 토큰화 결과
tokens = tokenizer.tokenize(text)
print(f"문장 길이 : {len(text)}")
print(f"토큰화된 결과: {tokens}")
print(f"토큰 개수: {len(tokens)}")

# 토큰 ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"토큰 ID: {token_ids}")

문장 길이 : 20
토큰화된 결과: ['▁안녕하세요', ',', '▁오늘', '▁날씨', '가', '▁참', '▁', '좋', '네요', '!']
토큰 개수: 10
토큰 ID: [107687, 4, 42742, 166687, 713, 42297, 6, 87648, 25861, 38]


In [21]:
sen_len = 0
token_len = 0

for i in result:
    sen_len += len(i.page_content)
    token_len += len(tokenizer.tokenize(i.page_content))
print(f"문장 길이: {sen_len}")
print(f"토큰 개수: {token_len}")
print()


문장 길이: 3393
토큰 개수: 1819



# re-ranker 테스트

In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Re-ranker 초기화
model_name = "Alibaba-NLP/gte-multilingual-reranker-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

def re_rank(query, docs):
    inputs = tokenizer([(query, doc) for doc in docs], 
                       padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = reranker(**inputs)
        scores = outputs.logits.squeeze(-1)
    ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
    return ranked_docs

# Chroma에서 검색된 문서 가져오기
query = "제가 다니던 회사가 갑작스럽게 폐업했는데, 아직 월급을 받지 못했습니다. 이런 경우 어떻게 해야 하나요?"
retrieved_docs = retriever.get_relevant_documents(query)

# Re-ranking
re_ranked_docs = re_rank(query, [doc.page_content for doc in retrieved_docs])

In [48]:
retrieved_docs

[Document(metadata={'article': '제36조', 'related_references': '제36조', 'revision_dates': '[{"year": 2020, "month": 5, "day": 26}]', 'source': 'file/현행법령_copy/근로기준법(법률)(제18176호)(20211119).pdf', 'title': '금품 청산'}, page_content='제36조(금품 청산) 사용자는 근로자가 사망 또는 퇴직한 경우에는 그 지급 사유가 발생한 때부터 14일 이내에 임금, 보\n상금, 그 밖의 모든 금품을 지급하여야 한다. 다만, 특별한 사정이 있을 경우에는 당사자 사이의 합의에 의하여 기일\n을 연장할 수 있다. <개정 2020. 5. 26.>'),
 Document(metadata={'article': '제281조', 'related_references': '제281조', 'revision_dates': 'Not Applicable', 'source': 'file/현행법령_copy/채무자 회생 및 파산에 관한 법률(법률)(제20264호)(99990101).pdf', 'title': '퇴직금 등'}, page_content='제281조(퇴직금 등) ①회생절차개시 후 채무자의 이사ㆍ대표이사ㆍ감사 또는 근로자이었던 자로서 계속하여 신회사의\n이사ㆍ대표이사ㆍ감사 또는 근로자가 된 자는 채무자에서 퇴직한 것을 이유로 하여 퇴직금 등을 지급받을 수 없다.\n②제1항에 규정된 자가 채무자에서 재직한 기간은 퇴직금 등의 계산에 관하여는 신회사에서 재직한 기간으로 본다.'),
 Document(metadata={'article': '제23조', 'related_references': '제43조, 제23조, 제27조, 제36조, 제51조, 제52조, 제56조, 제5조', 'revision_dates': '[{"year": 2021, "month": 3, "day": 30}]', 'source': 'file/현행법령_copy/근로기준법

In [49]:
[x.page_content for x in retrieved_docs]

['제36조(금품 청산) 사용자는 근로자가 사망 또는 퇴직한 경우에는 그 지급 사유가 발생한 때부터 14일 이내에 임금, 보\n상금, 그 밖의 모든 금품을 지급하여야 한다. 다만, 특별한 사정이 있을 경우에는 당사자 사이의 합의에 의하여 기일\n을 연장할 수 있다. <개정 2020. 5. 26.>',
 '제281조(퇴직금 등) ①회생절차개시 후 채무자의 이사ㆍ대표이사ㆍ감사 또는 근로자이었던 자로서 계속하여 신회사의\n이사ㆍ대표이사ㆍ감사 또는 근로자가 된 자는 채무자에서 퇴직한 것을 이유로 하여 퇴직금 등을 지급받을 수 없다.\n②제1항에 규정된 자가 채무자에서 재직한 기간은 퇴직금 등의 계산에 관하여는 신회사에서 재직한 기간으로 본다.',
 '제23조(매월 1회 이상 지급하여야 할 임금의 예외) 법 제43조제2항 단서에서 “임시로 지급하는 임금, 수당, 그 밖에 이\n에 준하는 것 또는 대통령령으로 정하는 임금”이란 다음 각 호의 것을 말한다.\n1. 1개월을 초과하는 기간의 출근 성적에 따라 지급하는 정근수당\n2. 1개월을 초과하는 일정 기간을 계속하여 근무한 경우에 지급되는 근속수당\n3. 1개월을 초과하는 기간에 걸친 사유에 따라 산정되는 장려금, 능률수당 또는 상여금\n4. 그 밖에 부정기적으로 지급되는 모든 수당\n제23조의2(체불사업주 명단 공개 제외 대상) 법 제43조의2제1항 단서에서 “체불사업주의 사망ㆍ폐업으로 명단 공개의\n실효성이 없는 경우 등 대통령령으로 정하는 사유”란 다음 각 호의 어느 하나에 해당하는 경우를 말한다. <개정\n2021. 3. 30.>\n1. 법 제36조, 제43조, 제51조의3, 제52조제2항제2호 또는 제56조에 따른 임금, 보상금, 수당, 그 밖의 일체의 금품\n(이하 “임금등”이라 한다)을 지급하지 않은 사업주(이하 “체불사업주”라 한다)가 사망하거나 「민법」 제27조에 따\n라 실종선고를 받은 경우(체불사업주가 자연인인 경우만 해당한다)\n2. 체불사업주가 법 제43조의2제2항에 따른 소명 기간 종료 전까지 

In [50]:
re_ranked_docs

['제67조(환급청구권) ① 상호회사에서 퇴사한 사원은 정관이나 보험약관으로 정하는 바에 따라 그 권리에 따른 금액의\n환급을 청구할 수 있다.\n② 퇴사한 사원이 회사에 대하여 부담한 채무가 있는 경우에는 회사는 제1항의 금액에서 그 채무액을 공제할 수 있\n다.',
 '제26조(해고의 예고) 사용자는 근로자를 해고(경영상 이유에 의한 해고를 포함한다)하려면 적어도 30일 전에 예고를 하\n여야 하고, 30일 전에 예고를 하지 아니하였을 때에는 30일분 이상의 통상임금을 지급하여야 한다. 다만, 다음 각 호\n의 어느 하나에 해당하는 경우에는 그러하지 아니하다. <개정 2010. 6. 4., 2019. 1. 15.>\n1. 근로자가 계속 근로한 기간이 3개월 미만인 경우\n2. 천재ㆍ사변, 그 밖의 부득이한 사유로 사업을 계속하는 것이 불가능한 경우\n3. 근로자가 고의로 사업에 막대한 지장을 초래하거나 재산상 손해를 끼친 경우로서 고용노동부령으로 정하는 사유\n에 해당하는 경우',
 '제43조(퇴직판정의 특례) ① 법 제22조제1항제2호를 적용할 때 다음 각 호의 어느 하나에 해당하는 사유가 발생했으나\n퇴직급여를 실제로 받지 않은 경우는 퇴직으로 보지 않을 수 있다. <개정 2015. 2. 3., 2020. 2. 11.>\n1. 종업원이 임원이 된 경우\n2. 합병ㆍ분할 등 조직변경, 사업양도, 직ㆍ간접으로 출자관계에 있는 법인으로의 전출 또는 동일한 사업자가 경영\n하는 다른 사업장으로의 전출이 이루어진 경우\n3. 법인의 상근임원이 비상근임원이 된 경우\n4. 비정규직 근로자(「기간제 및 단시간근로자 보호 등에 관한 법률」에 따른 기간제근로자 또는 단시간근로자를 말\n한다)가 정규직 근로자(「근로기준법」에 따라 근로계약을 체결한 근로자로서 비정규직 근로자가 아닌 근로자를\n말한다)로 전환된 경우\n② 계속근로기간 중에 다음 각 호의 어느 하나에 해당하는 사유로 퇴직급여를 미리 지급받은 경우(임원인 근로소득\n자를 포함하며, 이하 “퇴직소득중간지급”이라

# retriever, re-ranker lm studio 연결 테스트

In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer


# Re-ranker 클래스 정의
class ReRanker:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

    def re_rank(self, query, docs):
        inputs = self.tokenizer([(query, doc) for doc in docs], 
                                padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.reranker(**inputs)
            scores = outputs.logits.squeeze(-1)
        ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
        return ranked_docs


# SentenceTransformer 임베딩 클래스 정의
class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()


# Chroma 기반 Retriever 초기화
def load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", k=10):
    # Initialize the embedding model
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    
    # Load Chroma vectorstore with the embedding function
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    
    # Create retriever
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    
    return retriever


# Chroma 디렉토리 및 모델 설정
chroma_dir = "chroma_data_folder_final"
embedding_model_name = "jinaai/jina-embeddings-v3"
reranker_model_name = "Alibaba-NLP/gte-multilingual-reranker-base"

# Retriever와 Re-ranker 초기화
retriever = load_chroma_retriever_with_embedding(chroma_dir, embedding_model_name, k=10)
reranker = ReRanker(reranker_model_name)

# Query 정의
query = "제가 다니던 회사가 갑작스럽게 폐업했는데, 아직 월급을 받지 못했습니다. 이런 경우 어떻게 해야 하나요?"

# Retrieve documents
retrieved_docs = retriever.get_relevant_documents(query)

# Re-rank documents
re_ranked_docs = reranker.re_rank(query, [doc.page_content for doc in retrieved_docs])

# 결과 출력
print("Re-ranked documents:")
for idx, doc in enumerate(re_ranked_docs):
    print(f"{idx+1}. {doc}")

Re-ranked documents:
1. 제67조(환급청구권) ① 상호회사에서 퇴사한 사원은 정관이나 보험약관으로 정하는 바에 따라 그 권리에 따른 금액의
환급을 청구할 수 있다.
② 퇴사한 사원이 회사에 대하여 부담한 채무가 있는 경우에는 회사는 제1항의 금액에서 그 채무액을 공제할 수 있
다.
2. 제26조(해고의 예고) 사용자는 근로자를 해고(경영상 이유에 의한 해고를 포함한다)하려면 적어도 30일 전에 예고를 하
여야 하고, 30일 전에 예고를 하지 아니하였을 때에는 30일분 이상의 통상임금을 지급하여야 한다. 다만, 다음 각 호
의 어느 하나에 해당하는 경우에는 그러하지 아니하다. <개정 2010. 6. 4., 2019. 1. 15.>
1. 근로자가 계속 근로한 기간이 3개월 미만인 경우
2. 천재ㆍ사변, 그 밖의 부득이한 사유로 사업을 계속하는 것이 불가능한 경우
3. 근로자가 고의로 사업에 막대한 지장을 초래하거나 재산상 손해를 끼친 경우로서 고용노동부령으로 정하는 사유
에 해당하는 경우
3. 제43조(퇴직판정의 특례) ① 법 제22조제1항제2호를 적용할 때 다음 각 호의 어느 하나에 해당하는 사유가 발생했으나
퇴직급여를 실제로 받지 않은 경우는 퇴직으로 보지 않을 수 있다. <개정 2015. 2. 3., 2020. 2. 11.>
1. 종업원이 임원이 된 경우
2. 합병ㆍ분할 등 조직변경, 사업양도, 직ㆍ간접으로 출자관계에 있는 법인으로의 전출 또는 동일한 사업자가 경영
하는 다른 사업장으로의 전출이 이루어진 경우
3. 법인의 상근임원이 비상근임원이 된 경우
4. 비정규직 근로자(「기간제 및 단시간근로자 보호 등에 관한 법률」에 따른 기간제근로자 또는 단시간근로자를 말
한다)가 정규직 근로자(「근로기준법」에 따라 근로계약을 체결한 근로자로서 비정규직 근로자가 아닌 근로자를
말한다)로 전환된 경우
② 계속근로기간 중에 다음 각 호의 어느 하나에 해당하는 사유로 퇴직급여를 미리 지급받은 경우(임원인 근로소득
자를 포함하며, 이하 “퇴직소득중간지급”이

# 프롬프트 엔지니어링

In [ ]:
import torch
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

RAG_PROMPT_TEMPLATE = (
"""
You are a chatbot that provides answers exclusively based on the 'Context' below. 
Do not add or infer laws, articles, or details not explicitly found in the Context. 
If the Context does not provide enough information to answer accurately, respond with:
"제가 제공할 수 있는 정보가 없습니다."

When explaining legal content:
- Restate or simplify the provided legal language into clear, user-friendly terms.
- Show only the most relevant references from the Context. Omit unrelated or partial references.
- Keep your explanation concise and focused. Avoid over-long text or unnecessary details.
- If a specific law or article is mentioned in the Context, cite it as stated (include the law name and article/paragraph if available).
- If there is no clear legal reference in the Context, indicate there is no explicit reference.

Question: {question}
Context: {context}
Answer:
"""
)

SYS_PROMPT_TEMPLATE = (
"""
당신은 대한민국 현행 법령을 기반으로 작동하는 RAG(ChatGPT) 챗봇입니다.

- 아래 컨텍스트(Context)에 제시된 법령 정보만 사용하세요.
- 맥락과 맞지 않는 법령, 혹은 불완전하거나 부분적인 내용은 과감히 배제하세요.
- 법령 조항을 그대로 복사하기보다, 사용자 친화적이도록 쉽고 간결하게 재해석하십시오.
- 필요한 최소한의 핵심 정보만 담되, 법령 명칭과 조항 번호는 정확히 표시하세요.
- 답변이 지나치게 길어지지 않도록 주의하세요.
- 답변에는 실제 사용한 법령과 조항을 짧게 기재하세요.
- 만약 컨텍스트에 충분한 정보가 없다면 "제가 제공할 수 있는 정보가 없습니다."라고만 답변하십시오.

위 지침을 엄격히 준수하여 답변을 생성하세요.
"""
)

class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()
    

# Re-ranker 클래스 정의
class ReRanker:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

    def re_rank(self, query, docs):
        inputs = self.tokenizer([(query, doc) for doc in docs], 
                                padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.reranker(**inputs)
            scores = outputs.logits.squeeze(-1)
        ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
        return ranked_docs


def load_chroma_retriever_with_embedding_and_reranker(
    chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", reranker_model_name = "Alibaba-NLP/gte-multilingual-reranker-base", k=10
):
    # Initialize the embedding model
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    
    # Load Chroma vectorstore with the embedding function
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    
    # Create retriever
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    
    # Initialize the re-ranker
    reranker = ReRanker(reranker_model_name)
    
    return retriever, reranker


def setup_rag_system_with_reranker(llm_base_url, embedding_model_name="jinaai/jina-embeddings-v3", reranker_model_name = "Alibaba-NLP/gte-multilingual-reranker-base"):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", SYS_PROMPT_TEMPLATE),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()
    
    # Load retriever and re-ranker
    retriever, reranker = load_chroma_retriever_with_embedding_and_reranker(
        chroma_dir="chroma_data_folder_final_copy",
        embedding_model_name=embedding_model_name,
        reranker_model_name=reranker_model_name,
        k=10
    )

    return llm, rag_chain, retriever, reranker


def run_pipeline(question, llm_base_url="http://localhost:1234/v1", chroma_dir="chroma_data_folder_final_copy", reranker_model_name = "Alibaba-NLP/gte-multilingual-reranker-base"):
    # Load system components
    llm, rag_chain, retriever, reranker = setup_rag_system_with_reranker(
        llm_base_url=llm_base_url, reranker_model_name=reranker_model_name
    )
    
    # Retrieve documents
    retrieved_docs = retriever.get_relevant_documents(question)
    
    # Re-rank documents
    ranked_docs = reranker.re_rank(question, [doc.page_content for doc in retrieved_docs])
    
    # Combine ranked documents into a single context
    context = " ".join(ranked_docs)
    
    # Generate response
    response = rag_chain.invoke({"question": question, "context": context})
    
    return response, retrieved_docs, ranked_docs


# Example usage
chroma_dir = "chroma_data_folder_final_copy"  # Path to the Chroma directory
api_url = "http://localhost:1234/v1"  # Local OpenAI server URL
api_key = "lm-studio"  # API key for local OpenAI server
model_name = "teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf"

print("끝")

/Users/shinminseog/opt/anaconda3/envs/rag_env/lib/python3.9/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


끝


In [3]:
question = "명예훼손과 모욕죄의 차이는 무엇인가요?"

In [ ]:
result, retrieved_docs, ranked_docs = run_pipeline(question)
print(result)

TypeError: __init__() got an unexpected keyword argument 'trust_remote_code'

In [2]:
questions = [
    "운전 중 휴대전화를 사용하는 것이 법적으로 금지되어 있나요?",
    "산업재해가 발생했을 때 회사가 해야 할 첫 번째 조치는 무엇인가요?",
    "아동을 대상으로 한 광고는 어떤 규제를 받나요?",
    "휴가 중 업무용 연락을 받는 것은 법적으로 문제가 되나요?",
    "임차인이 계약 만료 후에도 주택에 계속 거주할 수 있는 조건은 무엇인가요?",
    "제가 다니던 회사가 갑작스럽게 폐업했는데, 아직 월급을 받지 못했습니다. 이런 경우 어떻게 해야 하나요?",
    "아파트 층간소음 문제로 윗집과 심각한 갈등이 있습니다. 법적으로 해결할 방법이 있나요?",
    "제가 온라인에서 구매한 물건이 광고와 다르게 도착했는데, 환불이 가능한가요?",
    "임대인이 보증금을 반환하지 않고 이사비용을 요구합니다. 법적으로 어떻게 대응해야 하나요?",
    "길에서 강아지를 주웠는데, 주인을 찾지 못하면 제가 키워도 되나요?",
    "회사가 임신 중인 직원을 해고했습니다. 법적으로 대응할 수 있는 방법이 있나요?",
    "친구에게 빌려준 돈을 돌려받지 못했는데, 법적으로 강제할 수 있나요?",
    "제가 길에서 교통사고를 당했는데, 상대방이 보험 처리를 거부합니다. 어떻게 해야 하나요?"
]

In [3]:
result_list = []

for question in questions:
    result, retriever_result, ranked_docs = run_pipeline(question)
    result_list.append(result)

/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_16583/1360761439.py:79: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
/var/folders/2w/p0zgpvps62gb8jlzgy_v7zgh0000gn/T/ipykernel_16583/1360761439.py:124: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


KeyboardInterrupt: 

# 디버깅 하며 테스트

In [3]:
import torch
import gc
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Debug 시작
print("✅ DEBUG START: Initializing RAG System")


RAG_PROMPT_TEMPLATE = (
"""
You are a chatbot that provides answers exclusively based on the 'Context' below. 
Do not add or infer laws, articles, or details not explicitly found in the Context. 
If the Context does not provide enough information to answer accurately, respond with:
"제가 제공할 수 있는 정보가 없습니다."

When explaining legal content:
- Restate or simplify the provided legal language into clear, user-friendly terms.
- Show only the most relevant references from the Context. Omit unrelated or partial references.
- Keep your explanation concise and focused. Avoid over-long text or unnecessary details.
- If a specific law or article is mentioned in the Context, cite it as stated (include the law name and article/paragraph if available).
- If there is no clear legal reference in the Context, indicate there is no explicit reference.

Question: {question}
Context: {context}
Answer:
"""
)

SYS_PROMPT_TEMPLATE = (
"""
당신은 대한민국 현행 법령을 기반으로 작동하는 RAG(ChatGPT) 챗봇입니다.

- 아래 컨텍스트(Context)에 제시된 법령 정보만 사용하세요.
- 맥락과 맞지 않는 법령, 혹은 불완전하거나 부분적인 내용은 과감히 배제하세요.
- 법령 조항을 그대로 복사하기보다, 사용자 친화적이도록 쉽고 간결하게 재해석하십시오.
- 필요한 최소한의 핵심 정보만 담되, 법령 명칭과 조항 번호는 정확히 표시하세요.
- 답변이 지나치게 길어지지 않도록 주의하세요.
- 답변에는 실제 사용한 법령과 조항을 짧게 기재하세요.
- 만약 컨텍스트에 충분한 정보가 없다면 "제가 제공할 수 있는 정보가 없습니다."라고만 답변하십시오.

위 지침을 엄격히 준수하여 답변을 생성하세요.
"""
)


class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        print(f"🔍 Initializing SentenceTransformer with model: {model_name}")
        try:
            gc.collect()  # ✅ 메모리 정리
            self.model = SentenceTransformer(model_name, trust_remote_code=True)
            print("✅ SentenceTransformer 로드 성공")
        except Exception as e:
            print(f"❌ SentenceTransformer 로드 실패: {e}")
            raise

    def embed_documents(self, texts):
        print(f"🔍 Embedding {len(texts)} documents")
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        print(f"🔍 Embedding query: {text}")
        return self.model.encode([text])[0].tolist()


class ReRanker:
    def __init__(self, model_name):
        print(f"🔍 Initializing ReRanker with model: {model_name}")
        try:
            gc.collect()  # ✅ 모델 로드 전에 메모리 정리
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            self.reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True)

            # MPS 사용 가능하면 MPS로 이동, 불가능하면 CPU 사용
            self.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
            self.reranker.to(self.device)

            print("✅ ReRanker 로드 성공 (Using device:", self.device, ")")
        except Exception as e:
            print(f"❌ ReRanker 로드 실패: {e}")
            raise

    def re_rank(self, query, docs):
        print(f"🔍 Re-ranking {len(docs)} documents for query: {query}")

        if not docs:
            print("❗ Re-ranker: 문서가 없습니다.")
            return []

        try:
            gc.collect()  # ✅ 문서 정렬 전 메모리 정리
            inputs = self.tokenizer([(query, doc) for doc in docs], 
                                    padding=True, truncation=True, return_tensors='pt').to(self.device)

            with torch.no_grad():
                outputs = self.reranker(**inputs)
                scores = outputs.logits.squeeze(-1)

            ranked_docs = [doc for _, doc in sorted(zip(scores.cpu(), docs), reverse=True)]
            print("✅ Re-ranking 완료")

            gc.collect()  # ✅ 문서 정렬 후 메모리 정리
            return ranked_docs
        except Exception as e:
            print(f"❌ Re-ranking 실패: {e}")
            raise


def load_chroma_retriever_with_embedding_and_reranker(
    chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", reranker_model_name="Alibaba-NLP/gte-multilingual-reranker-base", k=10
):
    print("🔍 Loading Chroma retriever...")

    try:
        gc.collect()  # ✅ Chroma 로드 전 메모리 정리
        embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    except Exception as e:
        print(f"❌ Embedding 모델 로드 실패: {e}")
        raise

    try:
        vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
        retriever = vectorstore.as_retriever(search_kwargs={'k': k})
        print("✅ Chroma retriever 로드 성공")
    except Exception as e:
        print(f"❌ ChromaDB 로드 실패: {e}")
        raise

    try:
        reranker = ReRanker(reranker_model_name)
    except Exception as e:
        print(f"❌ ReRanker 로드 실패: {e}")
        raise

    return retriever, reranker


def setup_rag_system_with_reranker(llm_base_url, embedding_model_name="jinaai/jina-embeddings-v3", reranker_model_name="Alibaba-NLP/gte-multilingual-reranker-base"):
    print("🔍 Setting up RAG System...")
    
    try:
        gc.collect()  # ✅ RAG 시스템 설정 전 메모리 정리
        llm = ChatOpenAI(
            base_url=llm_base_url,
            api_key="lm-studio",
            model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
            temperature=0.3,
        )
        print("✅ LLM 로드 성공")
    except Exception as e:
        print(f"❌ LLM 로드 실패: {e}")
        raise

    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", SYS_PROMPT_TEMPLATE),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    retriever, reranker = load_chroma_retriever_with_embedding_and_reranker(
        chroma_dir="chroma_data_folder_final_copy",
        embedding_model_name=embedding_model_name,
        reranker_model_name=reranker_model_name,
        k=10
    )

    return llm, rag_chain, retriever, reranker


def run_pipeline(question, llm_base_url="http://localhost:1234/v1", chroma_dir="chroma_data_folder_final_copy", reranker_model_name="Alibaba-NLP/gte-multilingual-reranker-base"):
    print(f"\n🔍 Running pipeline for question: {question}")

    try:
        gc.collect()  # ✅ 파이프라인 실행 전 메모리 정리
        llm, rag_chain, retriever, reranker = setup_rag_system_with_reranker(
            llm_base_url=llm_base_url, reranker_model_name=reranker_model_name
        )
    except Exception as e:
        print(f"❌ RAG 시스템 초기화 실패: {e}")
        raise

    try:
        retrieved_docs = retriever.invoke(question)
        print(f"✅ Retrieved {len(retrieved_docs)} documents")
        gc.collect()  # ✅ 검색 후 메모리 정리
    except Exception as e:
        print(f"❌ Document retrieval 실패: {e}")
        raise

    try:
        ranked_docs = reranker.re_rank(question, [doc.page_content for doc in retrieved_docs])
        print(f"✅ Ranked {len(ranked_docs)} documents")
    except Exception as e:
        print(f"❌ Document ranking 실패: {e}")
        raise

    context = " ".join(ranked_docs)

    try:
        response = rag_chain.invoke({"question": question, "context": context})
        print("✅ LLM 응답 생성 완료")
        return response, retrieved_docs, ranked_docs
    except Exception as e:
        print(f"❌ LLM 응답 생성 실패: {e}")
        raise


# 실행 예시
question = "명예훼손과 모욕죄의 차이는 무엇인가요?"
result, retrieved_docs, ranked_docs = run_pipeline(question)
print(result)

✅ DEBUG START: Initializing RAG System

🔍 Running pipeline for question: 명예훼손과 모욕죄의 차이는 무엇인가요?
🔍 Setting up RAG System...
✅ LLM 로드 성공
🔍 Loading Chroma retriever...
🔍 Initializing SentenceTransformer with model: jinaai/jina-embeddings-v3
✅ SentenceTransformer 로드 성공
✅ Chroma retriever 로드 성공
🔍 Initializing ReRanker with model: Alibaba-NLP/gte-multilingual-reranker-base
✅ ReRanker 로드 성공 (Using device: mps )
🔍 Embedding query: 명예훼손과 모욕죄의 차이는 무엇인가요?
✅ Retrieved 10 documents
🔍 Re-ranking 10 documents for query: 명예훼손과 모욕죄의 차이는 무엇인가요?
✅ Re-ranking 완료
✅ Ranked 10 documents
✅ LLM 응답 생성 완료

명예훼손과 모욕죄는 모두 다른 사람의 명성을 해치는 행위에 해당하지만, 그 성격과 처벌이 다릅니다.

명예훼손(형법 제307조)은 사실을 드러내거나 거짓 정보를 퍼뜨려 누군가의 명예를 훼손하는 행위를 말합니다. 공연히 허위 사실을 퍼뜨린 경우 최대 2년의 징역이나 금고 또는 500만 원 이하의 벌금에 처할 수 있습니다.

반면 모욕(형법 제311조)은 사람을 비하하거나 모욕적인 언어를 사용하여 그 사람의 명예를 해치는 행위를 말합니다. 공연히 사람을 모욕한 경우 최대 1년 이하의 징역이나 금고 또는 200만 원 이하의 벌금에 처할 수 있습니다.

요약하자면, 명예훼손은 사실적 허위 정보를 퍼뜨려 누군가의 명성을 해치는 행위이고, 모욕은 비하하는 언어를 사용하여 그 사람의 명성을 해치는 행위입니다.
